# Chicago Bike Sharing Exploration

## Set-up

First we import the required libraries.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta

Then we read in the dataset of the Chicago bike-sharing company of the year 2018.

In [ ]:
data_messy = pd.read_csv("chicago_2018.csv")

## Some first insights

I want to see some first samples:

In [ ]:
data_messy.head(5)

Now I am interested into how many observations there are within the dataset.

In [ ]:
data_messy.count()

I want to know the time of the first and last measurement.

In [ ]:
min = data_messy["start_time"].min()
max = data_messy["end_time"].max()

print("The measurements took place from", min, "until", max + ".")

How many *start* and *end* stations are there?

In [ ]:
start_stations = list(data_messy["start_station_id"].unique())
end_stations = list(data_messy["end_station_id"].unique())
print("There are", len(start_stations), "start stations, and", len(end_stations), "end stations")

Thus there are 2 more start stations than end stations. This might be because there are stations,
* in which you can only rent a bike; not return
* which have multiple IDs
* which are only for maintenance purposes

Or because
* start station IDs and end station IDs are completely uncoupled and one of the reasons above

I was wondering how big the fleet is:

In [ ]:
def getTime(str):
    return datetime.strptime(str, '%Y-%m-%d %H:%M:%S')

In [ ]:
num_bikes = len(list(data_messy["bike_id"].unique()))

print("The fleet consists of", num_bikes, "bikes.")

What is the ratio of subscribed users as compared to all users in 2018?

In [ ]:
num_subs = len(data_messy[data_messy["user_type"] == "Subscriber"])
num_tot = len(data_messy)

print(round((num_subs / num_tot * 100), 2), "% of users are actual subscribers")

Check for null values.

In [ ]:
data_messy[data_messy["user_type"].isnull()]

Seemingly no null values in the dataset.

Engineering a new feature "duration":

In [ ]:
data_messy["duration"] = (pd.to_datetime(data_messy["end_time"]) - pd.to_datetime(data_messy["start_time"]))

In [ ]:
data_messy.head()

In [ ]:
data_messy["duration"].sort_values().head(10)

7 subsequent observations have a negative duration. Looking at the dates, it becomes obvious, that the underlying reason is the time change:

In [ ]:
data_messy.loc[2946878, "start_time"]

In [ ]:
data_messy.loc[2946878, "end_time"]

You can find information about the **negative** time change (summer to winter time on the 4th of November 2018: 2 PM to 1 PM) [here](https://www.timeanddate.de/stadt/zeitumstellung/usa/chicago).

It may be necessary to adjust values for the other **positive** time change (winter to summer on the 11th of May 2018: 2 PM to 3 PM).

How high is the maximum and minimum utilization of the fleet?

First I need to define a function which converts the string into a datetime format.

Depending on your machine, the following operations might take a while. Eventually it will terminate...

In [ ]:
time_bike = np.empty(6500)

for index, row in data_messy.iterrows():
    time_bike[row["bike_id"]] = time_bike[row["bike_id"]] + ((getTime(row["end_time"]) - getTime(row["start_time"])).total_seconds() / 3600)

In [ ]:
time_bike_series = pd.DataFrame(time_bike, columns = ["Utilization"])

In [ ]:
time_bike_series.describe()

In [ ]:
time_bike_series.dropna(inplace = True)

In [ ]:
time_bike_series.tail()

In [ ]:
time_bike_series.describe()

In [ ]:
f, axes = plt.subplots(1, sharey=True, figsize=(15, 15))

sns.boxplot(data = time_bike_series, palette = "magma")
plt.show()

Which bike was used the most?

In [ ]:
time_bike_series[time_bike_series["Utilization"] >= 4000]

Average utilization: (The average utilization will be way higher as there are also alot of null values included in this aggregation)

In [ ]:
av_ut = float((time_bike_series.mean() / (24 * 365))*100)

print("The average utilization is:", av_ut, "%")

Which stations are used the most?

In [ ]:
sns.countplot(x = "start_station_id", data = data_messy)
plt.show()

Which station is used the most?

In [ ]:
station_mostused = data_messy["start_station_id"].value_counts().idxmax()
print("The most used start station is the station with id", station_mostused)

# An attempt to visualize the most used stations geographically

First I import a dataset that I found on the [internet](https://data.cityofchicago.org/Transportation/Divvy-Bicycle-Stations-In-Service/67g3-8ig8) to describe the position (long, lat) of the stations.

In [ ]:
data_stations = pd.read_csv("Divvy_Bicycle_Stations_-_In_Service.csv")

In [ ]:
data_stations.head(10)

Joining (merging) the initial dataset with the dataset, which contains the coordinates for the stations on the column 'start_station_id', respectively 'ID'

In [ ]:
merged_data = pd.merge(data_messy, data_stations, left_on = "start_station_id", right_on = "ID")

In [ ]:
merged_data.head(3)

Adding a new column "Coordinates" because the location, lat' and long' variables cannot be simply plotted by folium.

In [ ]:
merged_data["Coordinates"] = list(zip(merged_data["Latitude"].round(4), merged_data["Longitude"].round(4)))

In [ ]:
merged_data.head(3)

## Importing the relevant libraries:

In [ ]:
from pandas.plotting import register_matplotlib_converters
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
import timeit

# First visualization attempts

I will select a random bike for showing the locations to avoid overplotting and to avoid performance issues

In [ ]:
list_bikes = data_messy["bike_id"].unique()
random_bike = random.choice(list_bikes)
print(random_bike)

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] == random_bike]["Coordinates"], radius = 15))

heat_map

Sclicing the dataframe in order to be able to assign the correct popup to the markers

In [ ]:
station_location = merged_data[['start_station_id', 'Latitude', 'Longitude']].copy()
station_location = station_location.drop_duplicates(subset = ["start_station_id"])
station_location.head(10)

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] == random_bike]["Coordinates"], radius = 15))

for indice, row in station_location.iterrows():
    folium.CircleMarker(
        radius = 1,
        location=[row["Latitude"], row["Longitude"]],
        popup=row['start_station_id'],
        color='crimson', 
        fill_color='crimson'
    ).add_to(heat_map)
  
heat_map

Plotting the density of used bikestations for bikes with an id lower than 1000 (because plotting this for all bikes would just not terminate)

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] < 1000]["Coordinates"], radius = 15))

heat_map

Adding a column 'end_hour' to the dataframe, in order to be able to easily access only the hour from the 'end_time' timestamp

In [ ]:
merged_data['end_time'] = pd.to_datetime(merged_data['end_time'])
merged_data["end_hour"] = list(merged_data["end_time"].dt.hour)
merged_data.head()

Now taking all rows with an end time between 6am and 10am and assigning it to a new dataframe 

In [ ]:
morning_stations = merged_data[(merged_data["end_hour"]<=9) & (merged_data["end_hour"] >= 6)]
morning_stations.head()

Now taking all rows with an end time between 4pm and 7pm and assigning it to a new dataframe 

In [ ]:
evening_stations = merged_data[(merged_data["end_hour"]<=18) & (merged_data["end_hour"] >= 16)]
evening_stations.head()

Now we are able to **map all the stations, where people frequently drive to in the morning hours**. These stations will probably be in close proximity to common workplaces

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(morning_stations[morning_stations["bike_id"]==random_bike]["Coordinates"], radius = 15))

for indice, row in station_location.iterrows():
    folium.CircleMarker(
        radius = 1,
        location=[row["Latitude"], row["Longitude"]],
        popup=row['start_station_id'],
        color='crimson', 
        fill_color='crimson'
    ).add_to(heat_map)

heat_map

It seems that there is a clear aggregation of trips to the **central station** in the **morning hours**

Now we do the same for the **evening hours** , i.e. the end stations people drive to after work

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(evening_stations[evening_stations["bike_id"]==random_bike]["Coordinates"], radius = 15))

for indice, row in station_location.iterrows():
    folium.CircleMarker(
        radius = 1,
        location=[row["Latitude"], row["Longitude"]],
        popup=row['start_station_id'],
        color='crimson', 
        fill_color='crimson'
    ).add_to(heat_map)

heat_map

Now it is harder to identify a clear direction to where trips are typically going

We now try to identify the movement of the selected bike. We therefore draw the stations and trajectories.

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

merged_data_select = merged_data[merged_data["bike_id"]==random_bike]

heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] == random_bike]["Coordinates"], radius = 15))

for indice, row in station_location.iterrows():
    folium.CircleMarker(
        radius = 1,
        location=[row["Latitude"], row["Longitude"]],
        popup=row['start_station_id'],
        color='crimson', 
        fill_color='crimson'
    ).add_to(heat_map)

folium.PolyLine(merged_data_select["Coordinates"],color="blue", weight=2, opacity=0.8, ).add_to(heat_map)
    
heat_map

# Geographical Findings

## Communities with most Divvy Bikes traffic

![test](Chicago_community_areas_map.svg.png)

* all of Center

## Locations / POI with most traffic

In some zoom-scales (and the given radius) one can identify the following highly used stations:

* [Ogilvie Transportation Center](https://en.wikipedia.org/wiki/Ogilvie_Transportation_Center)
* [Chicago Union Station](https://en.wikipedia.org/wiki/Chicago_Union_Station)
* Jane Addams Memorial, see [googlemaps](https://www.google.de/maps/place/Divvy/@41.8922479,-87.6121172,88m/data=!3m1!1e3!4m13!1m7!3m6!1s0x880e2c3cd0f4cbed:0xafe0a6ad09c0c000!2sChicago,+IL,+USA!3b1!8m2!3d41.8781136!4d-87.6297982!3m4!1s0x880e2b53a71cd513:0xd04bc26796dcb054!8m2!3d41.892278!4d-87.612043) for position


# Attempt to take POI into consideration

The list of POI was found on [Wikipedia](https://en.wikipedia.org/wiki/List_of_Chicago_Landmarks) and can be downloaded under this [Link](https://tools.wmflabs.org/kmlexport?article=List_of_Chicago_Landmarks).

I had to reformat the file manually because of some issues that caused the file to not be read in properly by pandas.

In [ ]:
POI_data = pd.read_csv("convertcsvtest.csv")

In [ ]:
POI_data.head()

In [ ]:
POI_data.info()

In [ ]:
POI_data.head()

In [ ]:
POI_data["Coordinates"] = list(zip(POI_data["long"].round(4), POI_data["lat"].round(4)))

In [ ]:
POI_data.count()

In [ ]:
POI_data.head()

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] < 1000]["Coordinates"], radius = 15))

positions = list(POI_data["Coordinates"])
for position in positions:
    folium.CircleMarker(radius=1, location=position, popup='The Waterfront', 
                                 color='crimson', fill_color='crimson').add_to(heat_map)

heat_map

It seems that those POI's correspond nicely to the density of used stations.

## Finding out which stations are most popular

In [ ]:
data_stations.head()

In [ ]:
data_stations.info()

In [ ]:
data_stations.loc[2, "ID"]

In [ ]:
for i in range(693):
    data_stations.loc[i, "Usage"] = len(merged_data[merged_data["start_station_id"] == data_stations.loc[i, "ID"]])

In [ ]:
data_stations["Coordinates"] = list(zip(data_stations["Latitude"].round(4), data_stations["Longitude"].round(4)))

In [ ]:
data_stations.head()

In [ ]:
data_stations["Usage"].describe()

In [ ]:
sns.displot(data = data_stations["Usage"] , kde=True)

plt.show()

As expected a poisson-like distribution with a wide base of rather unpopular stations and a tiny share of extremely popular stations.

In [ ]:
len(data_messy["start_station_id"].unique())

In the following I attempt to plot the tops (reddish) and flops (blue) of stations in terms of popularity on a map.

In [ ]:
topTwy = data_stations[data_stations["ID"] <= 621].sort_values("Usage").tail(100)
lowTwy = data_stations[data_stations["ID"] <= 621].sort_values("Usage").head(100)

In [ ]:
topTwy["Popular"] = "crimson"
lowTwy["Popular"] = "blue"

topLowTwy = [topTwy, lowTwy]
topLowTwy = pd.concat(topLowTwy)
topLowTwy

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

for i in range(len(topLowTwy)):
    folium.CircleMarker(radius=1, location = topLowTwy.iloc[i]["Coordinates"], 
                                 color=topLowTwy.iloc[i]["Popular"], popup = topLowTwy.iloc[i]["ID"], fill_color='crimson').add_to(heat_map)

heat_map

As popular and unpopular stations nicely discriminate geographically, I'm interested as to how the entirety of stations would be ordered. 

In [ ]:
data_stations = data_stations[data_stations["ID"] < 621].sort_values("Usage")

g1 = data_stations[data_stations["Usage"] < 1000]
g2 = data_stations[(data_stations["Usage"] >= 1000) & (data_stations["Usage"] < 10000)]
g3 = data_stations[data_stations["Usage"] >= 10000]
                   
g1["Color"] = "green"
g2["Color"] = "yellow"
g3["Color"] = "red"

stat = [g1, g2, g3]
stat = pd.concat(stat)
stat

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

for i in range(len(stat)):
    folium.CircleMarker(radius=2, location = stat.iloc[i]["Coordinates"], 
                                 color=stat.iloc[i]["Color"], fill_color='crimson',
                                   popup = stat.iloc[i]["Usage"]).add_to(heat_map)

heat_map

### Sights:

* University of Chicago (in an area of unpopular stations, you can easily find the station as it is a red marker in a bunch of yellow markers in the south)

In [ ]:
data_messy[data_messy["start_station_id"] == 542].sort_values("start_time")

In [ ]:
stat[stat["ID"] == 542]["Usage"]

### Finding out when new stations were established

In [ ]:
data_messy[data_messy["start_station_id"] == 345]["start_time"].min()

In [ ]:
def getFirstDate(ID):
    return getTime(data_messy[data_messy["start_station_id"] == ID]["start_time"].min())

In [ ]:
getFirstDate(562)

In [ ]:
data_stations["First Use"] = data_stations["ID"].apply(lambda x: getFirstDate(x))

In [ ]:
data_stations.head()

In [ ]:
cutDate1 = pd.to_datetime("1st of February, 2018")
cutDate2 = pd.to_datetime("1st of May, 2018")

In [ ]:
data_stations.iloc[34]["First Use"] < cutDate1

In [ ]:
def categorize(x, a, b):
    if data_stations.iloc[x]["First Use"] < a:
        return "green"
    elif ((data_stations.iloc[x]["First Use"] >= a) & (data_stations.iloc[x]["First Use"] < b)):
        return "yellow"
    elif data_stations.iloc[x]["First Use"] > b:
        return "red"

In [ ]:
cutDate1 > cutDate2

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=9, control_scale=True, max_zoom=20)

for i in range(len(data_stations)):
    folium.CircleMarker(radius=2, location = data_stations.iloc[i]["Coordinates"], 
                                 color=categorize(i, cutDate1, cutDate2), fill_color='crimson',
                                   popup = data_stations.iloc[i]["Usage"]).add_to(heat_map)

heat_map

As there is a substancial amount of unpopular stations (i.e. being used less than 100 times in a year, some even less than 30), I was wondering what reason there might be. The establishment of new stations seems like a plausible idea.

Thus I plotted all stations in a color that encodes the time of year in which the station was used first (in this period of a year).

* green: station was used in January for the first time
* yellow: station was used in February for the first time
* red: station was used only after the begin of March

With the red stations being low in number this hypothisis is to be rejected. Also it is possible that even those are old stations which just haven't been used in the cold season.

## Considering auto traffic

Again some data I found on the [internet](https://www.chicago.gov/city/en/depts/cdot/dataset/average_daily_trafficcounts.html)

### Average Daily Traffic

Average Daily Traffic refers to the number of vehicles traveling through a particular point on the city streets in a 24-hour period. Average Daily Traffic (ADT) counts are analogous to a census count of vehicles on city streets. These counts provide a close approximation to the actual number of vehicles passing through a given location on an average weekday.

In [ ]:
traffic = pd.read_csv("Average_Daily_Traffic_Counts.csv", sep = ",")

In [ ]:
traffic["Coordinates"] = list(zip(traffic["Latitude"].round(4), traffic["Longitude"].round(4)))

In [ ]:
traffic.head()

In [ ]:
heat_map = folium.Map(location=(41.8695, -87.6555), tiles='OpenStreetMap', zoom_start=10, control_scale=True, max_zoom=20)

# heat_map.add_child(plugins.HeatMap(merged_data[merged_data["bike_id"] < 1000]["Coordinates"], radius = 25))

for i in range(0,len(traffic)):
   folium.Circle(
      location=[traffic.iloc[i]['Latitude'], traffic.iloc[i]['Longitude']],
      #popup=traffic.iloc[i]['Traffic Volume Count Location Address'],
      radius=float(traffic.iloc[i]['Total Passing Vehicle Volume'])*0.005,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(heat_map)

heat_map